# Get the Data of Same Opponent

In [1]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
nba_teams[:3]

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970}]

In [2]:
import json
with open('team_list.json', 'w') as f:
    json.dump(nba_teams, f, indent=4)

Read the data

In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
data_path = '../nba_shot/shot_data'
result_path = './opponent_data/'

In [5]:
# Get the file names in data path
fnames_data = os.listdir(data_path)

In [6]:
fpaths_data = []
for fname in fnames_data:
    fpath = os.path.join(data_path, fname)
    if os.path.isfile(fpath):
        fpaths_data.append(fpath)

In [7]:
print(fpaths_data[:3])

['../nba_shot/shot_data\\101107.csv', '../nba_shot/shot_data\\101108.csv', '../nba_shot/shot_data\\101123.csv']


Collect the data with same opponent

In [8]:
for fpath in fpaths_data[:]:
    df = pd.read_csv(fpath)
    
    # Clean the old data before 2015-2016 season
    df = df.drop(df[df['GAME_DATE']<20151000].index, axis=0)
    
    for team in nba_teams:
        df.loc[df['HTM'] == team['abbreviation'], 'HTM_ID'] = team['id']
        df.loc[df['VTM'] == team['abbreviation'], 'VTM_ID'] = team['id']

    df[['HTM_ID', 'VTM_ID']] = df[['HTM_ID', 'VTM_ID']].astype('int64')
    
    df.loc[df['HTM_ID']==df['TEAM_ID'], 'OPPONENT_TEAM_ID'] = df['VTM_ID']
    df.loc[df['HTM_ID']==df['TEAM_ID'], 'OPPONENT_TEAM_NAME'] = df['VTM']
    df.loc[df['HTM_ID']!=df['TEAM_ID'], 'OPPONENT_TEAM_ID'] = df['HTM_ID']
    df.loc[df['HTM_ID']!=df['TEAM_ID'], 'OPPONENT_TEAM_NAME'] = df['HTM']
    df['OPPONENT_TEAM_ID'] = df['OPPONENT_TEAM_ID'].astype('int64')

    for opponent in df['OPPONENT_TEAM_ID'].unique():
        fname = os.path.join(result_path, str(opponent)+'.csv')
        
        df_opponent = df[df['OPPONENT_TEAM_ID']==opponent]
        
        if not os.path.isfile(fname):
            df_opponent.to_csv(fname, mode='a', index=False, header=True)
        else:
            df_opponent.to_csv(fname, mode='a', index=False, header=False)